In [1]:
import re
import glob
import csv
from helpers import config
from helpers.loading import load_daily_data ,file_exist,get_all_dates
from helpers.algorithm import find_best_delay
import pandas as pd
import time
%load_ext autoreload
%autoreload 2

In [2]:
all_dates = get_all_dates(config["stock"])
print(f"{len(all_dates)} dates to process")

3034 dates to process


In [3]:
fieldnames = ['date', 'market1','market2',"lag"]

results_path = config["files"]["results"]["all_best_lags"] # file where to write the computed lags

result_file_exists = file_exist(results_path) 
csvfile = open(results_path, 'a', newline='') 
writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

if result_file_exists:
    processed_dates = set(pd.read_csv(results_path).date.unique())
else: 
    # if the file is new, we need to write headers
    writer.writeheader()
    processed_dates = set()


max_iterations = 500
start_time = time.time()
date_count = 0 # number of dates processed
for date_id,date in enumerate(all_dates[::10]):
    print(f"date:{date}, {date_id}:{len(all_dates)}, {100*date_id/len(all_dates):0.3f}%", end="\r")

    daily_data = load_daily_data(date,False)
    if not daily_data:
        # in case all markets do not provide data for the given date, we skip the date
        continue
        

    
    # we skip the current date if it has already been processed
    if date in processed_dates:
        continue
        
    for i,n1 in enumerate(daily_data):
        for j,n2 in enumerate(daily_data):
            if i>j: # avoid symetric (corr(a,b)=corr(b,a)) and meaningless (corr(a,a)=1) calculations
                best_delay, delays, correlations, los, his = find_best_delay(daily_data,n1,n2,step_size=1000)
                # write the computed result
                writer.writerow({'date': date, 'market1': n1,'market2': n2,'lag': best_delay})
                writer.writerow({'date': date, 'market1': n2,'market2': n1,'lag': -best_delay})
    csvfile.flush() # flush every time we processed a date
    date_count+=1
    if date_count>=max_iterations:
        break
print()
print(f"{date_count} dates processed in {time.time()-start_time:0.2f}s")
csvfile.close()

TypeError: unsupported operand type(s) for +: 'float' and 'str'

In [14]:
import numpy as np

In [29]:

pd.to_numeric(daily_data["NL"].price,errors="coerce").dropna()

date
2009-08-10 07:00:10.598000384+00:00    18.350
2009-08-10 07:00:10.601000192+00:00    18.350
2009-08-10 07:00:11.672999680+00:00    18.350
2009-08-10 07:00:11.993999616+00:00    18.350
2009-08-10 07:00:12.796000+00:00       18.345
                                        ...  
2009-08-10 15:29:56.794999808+00:00    18.220
2009-08-10 15:29:57.101000448+00:00    18.220
2009-08-10 15:29:59.256000256+00:00    18.220
2009-08-10 15:29:59.257999872+00:00    18.220
2009-08-10 15:35:03.772000256+00:00    18.210
Name: price, Length: 2788, dtype: float64

In [9]:
def generate_delayed_data(s1, s2, delay, join_type="outer"):
    s1 = s1.copy()
    s2 = s2.copy()
    
    h = min(s1.index.max(),s2.index.max())
    l = max(s1.index.min(),s2.index.min())
    print(h,l)
    s1.index = s1.index + pd.Timedelta(milliseconds=delay)
    pair_data = s1.join(s2, how=join_type, lsuffix="_1",
                        rsuffix="_2").ffill().dropna()
    return pair_data[(pair_data.index>=l)*(pair_data.index<=h)]
generate_delayed_data(daily_data["US"],daily_data["GB"],0)

2008-12-24 12:29:44.477999360+00:00 2008-12-24 14:30:24.156000+00:00


,price_1,price_2
date,,


In [6]:
for i,n1 in enumerate(daily_data):
        for j,n2 in enumerate(daily_data):
            if i>j: # avoid symetric (corr(a,b)=corr(b,a)) and meaningless (corr(a,a)=1) calculations
                print(n1,n2)
                best_delay, delays, correlations, los, his = find_best_delay(daily_data,n1,n2,step_size=1000)

US GB


ValueError: x and y must have length at least 2.